In [ ]:
import os
import numpy as np
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
from io import BytesIO
from PIL import Image

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
#helper functions

# Load the saved model
model = tf.keras.models.load_model('./tf-gpu/my_model_finetuned.h5')

column_names = ['image_id'] + [
    "5_o_Clock_Shadow", "Arched_Eyebrows", "Attractive", "Bags_Under_Eyes", "Bald", "Bangs",
    "Big_Lips", "Big_Nose", "Black_Hair", "Blond_Hair", "Blurry", "Brown_Hair", "Bushy_Eyebrows",
    "Chubby", "Double_Chin", "Eyeglasses", "Goatee", "Gray_Hair", "Heavy_Makeup", "High_Cheekbones",
    "Male", "Mouth_Slightly_Open", "Mustache", "Narrow_Eyes", "No_Beard", "Oval_Face", "Pale_Skin",
    "Pointy_Nose", "Receding_Hairline", "Rosy_Cheeks", "Sideburns", "Smiling", "Straight_Hair",
    "Wavy_Hair", "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace",
    "Wearing_Necktie", "Young"
]

# Define a function to preprocess the input image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))  # Resize the image
    img_array = image.img_to_array(img)  # Convert the image to a numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = img_array / 255.0  # Normalize to [0, 1]
    return img_array

# Define categories
categories = ['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Male', 'Young']

#return image_paths
def give_paths(image_directory):
    results = []
    # Iterate over all files in the given directory
    for root, _, files in os.walk(image_directory):
        for file in files:
            # Check if the file is an image (based on common image extensions)
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                results.append(os.path.join(root, file))
    return results
    
#definition to get predictions for images in specified dir
def update_predictions(image_directory, verbose=False):
    # Create an empty list to store the results for 'Blond hair' and 'Young'
    results = []
    images_without_blond_hair = []  
    images_not_young = []  
    
    # Loop over each image in the directory and make predictions
    for img_filename in os.listdir(image_directory):
        if img_filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(image_directory, img_filename)
            
            # Preprocess the image
            img = preprocess_image(img_path)
            
            # Get predictions from the model
            predictions = model.predict(img,verbose=0)  # This will be a 2D array (1, 5), corresponding to the 5 categories
            
            # Translate predictions to dataframe values
            young_pred = predictions[0][4]  # Prediction for 'Young'
            
            # Determine Male and Young values based on threshold
            young_value = 1 if young_pred > 0.5 else -1
            
            # Find the index of the highest hair color prediction
            hair_predictions = predictions[0][:3]  # First three predictions correspond to hair colors (Black, Blond, Brown)
            max_hair_index = np.argmax(hair_predictions)  # Get the index of the highest prediction
            if max_hair_index == 1:
                blond_value = 1  # Blond hair is set to 1
            else:
                blond_value = -1
            
            # Append the prediction results to the list
            results.append([img_filename, blond_value, young_value])

            # store image paths
            if blond_value == -1:
                images_without_blond_hair.append(img_path)
            if young_value == -1:
                images_not_young.append(img_path)
                
            # Optionally print the progress if verbose=True
            if verbose:
                print(f"Processed: {img_filename} | Blond hair: {blond_value}, Young: {young_value}")
    
    # Create a DataFrame with the results
    df = pd.DataFrame(results, columns=['Image Filename', 'Blond hair', 'Young'])
    
    # Return the DataFrame with the predictions
    return df, images_without_blond_hair, images_not_young


# Function to display images in a grid (5 images per row)
def display_images(images_paths):
    # Number of images per row
    images_per_row = 5
    num_images = len(images_paths)
    
    # Calculate number of rows needed to display all images
    num_rows = math.ceil(num_images / images_per_row)
    
    # Create a figure for the grid
    fig, axes = plt.subplots(num_rows, images_per_row, figsize=(images_per_row * 3, num_rows * 3))
    
    # Flatten the axes array to make indexing easier
    axes = axes.flatten()
    
    # Loop over each image and display it in the grid
    for i, img_path in enumerate(images_paths):
        img = mpimg.imread(img_path)
        axes[i].imshow(img)
        axes[i].axis('off')  # Hide axes for better display
    
    # Hide any unused axes
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    
    # Adjust the layout to prevent overlap
    plt.tight_layout()
    plt.show()

# Function to convert an image array to a byte array
def image_to_bytes(img_path):
    """
    Converts an image from a file path to a byte array.
    
    Args:
        img_path (str): The path to the image file.
        
    Returns:
        bytes: The byte data of the image.
    """
    # Open the image using PIL
    img = Image.open(img_path)
    img = img.convert("RGBA")
    
    # Create a BytesIO buffer to store the image
    img_buffer = BytesIO()
    # Save the image in the buffer as PNG format
    img.save(img_buffer, format="PNG")
    img_buffer.seek(0)  # Rewind the buffer to the beginning
    
    return img_buffer.read()  # Return the byte data of the image

#giving scores to side-by-side images, save as csv-file
def interactive_display_images(original_images_path, transformed_images_path, png_true, sample_size, save_filename):
    """
    Displays images side by side (original and transformed) with sliders above each image pair.
    
    Args:
        original_images_path (str): Path to the directory containing original images.
        transformed_images_path (list): List of paths to transformed images.
        sample_size (int, optional): Number of images to sample from transformed_images_path. Defaults to None (all images).
    
    Returns:
        list: Collected slider values.
    """
    # Sample the transformed images if sample_size is provided
    if sample_size:
        transformed_images_path = random.sample(transformed_images_path, min(sample_size, len(transformed_images_path)))

    # Input value storage
    input_box_values = []
    
    # Create the interactive layout
    output = widgets.Output()
    input_boxes = []
    
    for i, transformed_img_path in enumerate(transformed_images_path):
        # Extract the base filename (e.g., 086706.jpg)
        if png_true == 1:
            if '_' in os.path.basename(transformed_img_path):
                original_img_path = os.path.join(
                original_images_path,
                os.path.basename(transformed_img_path).split('_')[0] + ".png"
                )
            else:
                original_img_path = os.path.join(
                    original_images_path,
                    os.path.basename(transformed_img_path).split('.')[0] + ".png"
                )
        if png_true == 0:
            if '_' in os.path.basename(transformed_img_path):
                original_img_path = os.path.join(
                original_images_path,
                os.path.basename(transformed_img_path).split('_')[0] + ".jpg"
                )
            else:
                original_img_path = os.path.join(
                    original_images_path,
                    os.path.basename(transformed_img_path).split('.')[0] + ".jpg"
                )        
        # Create the Image widgets with defined width and height
        image_widget1 = widgets.Image(value=image_to_bytes(original_img_path), format='png', width=150, height=150)
        image_widget2 = widgets.Image(value=image_to_bytes(transformed_img_path), format='png', width=150, height=150)

        # Create an input_box
        input_box = widgets.IntText(value=0, min=0, max=10, step=1, description='Value:')
        input_boxes.append(input_box)
        
        # Create a horizontal box layout to display the image, slider, and image
        box = widgets.HBox([image_widget1, input_box, image_widget2])

        # Display the box
        display(box)

    # Define a button to submit input_box
    def submit_input_box(b):
        nonlocal input_box_values
        input_box_values = [input_box.value for input_box in input_boxes]
        df_values = pd.DataFrame(input_box_values, columns=['Score'])

        # Save to a CSV file
        csv_filename = save_filename
        df_values.to_csv(csv_filename, index=False)
        
        # Now, display the collected values using the Output widget
        with output:
            clear_output(wait=True)
            print("values collected:", input_box_values)

    submit_button = widgets.Button(description="Submit")
    submit_button.on_click(submit_input_box)
       
    # Display the button below the sliders
    display(output)
    display(submit_button)

**Quantitative analysis**

In [ ]:
#accuracy for CelebA single transformation
#normally this is the number correct predictions made by the classifier. 
#Here the expected output for all images is "blond hair," and the classifier's prediction for each image is compared to this expected output.
# accuracy = Number of correct predictions/Total number of predictions

# Call the update_predictions function to get the DataFrame
df_filtered, CelebA_ST_images_without_blond_hair, _  = update_predictions("./celeba_Orig/reduced/images/single transformation/images", verbose=False)

# Count the number of items (rows) in the DataFrame
item_count = len(df_filtered)

# Print the count
print(f"Number of items in df_filtered: {item_count}")

# Compare the predicted attribute ('Blond_hair') with the expected attribute
correct_predictions = df_filtered['Blond hair'] == 1  
correct_count = correct_predictions.sum()  # Sum up all True values (1's)

# Print the count of correct predictions
print(f"Number of correct predictions: {correct_count}")

# Calculate accuracy
accuracy = correct_predictions.mean()

# Print the accuracy
print(f'Accuracy: {accuracy * 100:.2f}%')

#print images
#display_images(CelebA_ST_images_without_blond_hair)

In [ ]:
#accuracy for FFHQ single transformation

# Call the update_predictions function to get the DataFrame
df_filtered, FFHQ_ST_images_without_blond_hair, _  = update_predictions("./ffhq/reduced/images/single transformation/images",verbose=False)

# Count the number of items (rows) in the DataFrame
item_count = len(df_filtered)

# Print the count
print(f"Number of items in df_filtered: {item_count}")

# Compare the predicted attribute ('Blond_hair') with the expected attribute
correct_predictions = df_filtered['Blond hair'] == 1  
correct_count = correct_predictions.sum()  # Sum up all True values (1's)

# Print the count of correct predictions
print(f"Number of correct predictions: {correct_count}")

# Calculate accuracy
accuracy = correct_predictions.mean()

# Print the accuracy
print(f'Accuracy: {accuracy * 100:.2f}%')

#print images
#display_images(FFHQ_ST_images_without_blond_hair)

In [ ]:
#accuracy for CelebA multi transformation

# Call the update_predictions function to get the DataFrame
df_filtered,CelebA_MT_images_without_blond_hair,CelebA_MT_images_not_young = update_predictions("./celeba_Orig/reduced/images/multi transformation/images", verbose=False)

# Print the count
print(f"Number of items in df_filtered: {item_count}")

# Expected values are 1 for both "Blond hair" and "Young"
# Compare both predicted attributes ('Blond_hair' and 'Young') with the expected value
# creation of two boolean Series
correct_predictions_blond = df_filtered['Blond hair'] == 1   # Blond hair expected to be 1
correct_predictions_young = df_filtered['Young'] == 1        # Young expected to be 1

# Calculate accuracy for both Blond hair and Young being correctly predicted
correct_predictions = correct_predictions_blond & correct_predictions_young
correct_count = correct_predictions.sum()  # Sum up all True values (1's)

# Print the count of correct predictions
print(f"Number of correct predictions: {correct_count}")

# Calculate overall accuracy
accuracy = correct_predictions.mean()

# Print the accuracy
print(f'Accuracy: {accuracy * 100:.2f}%')

#print images
#print("CelebA_MT_images_without_blond_hair")
#display_images(CelebA_MT_images_without_blond_hair)

#print("CelebA_MT_images_without_blond_hair")
#display_images(CelebA_MT_images_without_blond_hair)

In [ ]:
#accuracy for ffhq multi transformation

# Call the update_predictions function to get the DataFrame
df_filtered,FFHQ_MT_images_without_blond_hair,FFHQ_MT_images_not_young = update_predictions("./ffhq/reduced/images/multi transformation/images", verbose=False)

# Print the count
print(f"Number of items in df_filtered: {item_count}")

# Expected values are 1 for both "Blond hair" and "Young"
# Compare both predicted attributes ('Blond_hair' and 'Young') with the expected value
# creation of two boolean Series
correct_predictions_blond = df_filtered['Blond hair'] == 1   # Blond hair expected to be 1
correct_predictions_young = df_filtered['Young'] == 1        # Young expected to be 1

# Calculate accuracy for both Blond hair and Young being correctly predicted
correct_predictions = correct_predictions_blond & correct_predictions_young
correct_count = correct_predictions.sum()  # Sum up all True values (1's)

# Print the count of correct predictions
print(f"Number of correct predictions: {correct_count}")

# Calculate overall accuracy -  computes the proportion of True values in the correct_predictions series
accuracy = correct_predictions.mean()

# Print the accuracy
print(f'Accuracy: {accuracy * 100:.2f}%')

#print images
#print("FFHQ_MT_images_without_blond_hair")
#display_images(FFHQ_MT_images_without_blond_hair)

#print("FFHQ_MT_images_without_blond_hair")
#display_images(FFHQ_MT_images_without_blond_hair)

In [ ]:
#accuracy side by side
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Define the accuracy results
data = {
    'Transformation': ['CelebA Single', 'FFHQ Single', 'CelebA Multi', 'FFHQ Multi'],
    'Accuracy (%)': [89.04, 86.44, 75.64, 70.84]  # These should be the accuracy values you calculated
}

# Create a DataFrame
df_accuracy = pd.DataFrame(data)

# Plotting the bar chart
plt.figure(figsize=(8, 6))
sns.barplot(x='Transformation', y='Accuracy (%)', data=df_accuracy, palette='Blues_d')
plt.title('Accuracy Comparison: CelebA vs FFHQ', fontsize=16)
plt.xlabel('Transformation Type', fontsize=14)
plt.ylabel('Accuracy (%)', fontsize=14)
plt.ylim(0, 100)  # Set y-axis limits from 0 to 100
plt.xticks(rotation=45, ha='right')  # Rotate x labels for better readability
plt.tight_layout()
plt.show()

# Display the table (this can be included in the paper if needed)
print("Accuracy Table:")
print(df_accuracy.to_string(index=False))

# If you want to save the table to a file
df_accuracy.to_csv('accuracy_comparison.csv', index=False)


**Qualitative analysis**

through visual inspection of the transformed images, assessing whether the generated attributes are consistent with the intended transformation (blond hair, young) while maintaining realism and preserving the other attributes. Perform Visual Consistency Score: Give each image a score from 1 to 10 based on how well the transformation matches the target attribute (blond hair, young).

Qualitative analysis - scoring

In [ ]:
#CelebA single transformation (original images side by side with transformed images, sample size 10) - evaluate blond hair
interactive_display_images("./celeba_Orig/images",give_paths("./celeba_Orig/reduced/images/single transformation/images"), 0, 2,"values_CelebA_ST1.csv")
#rightclick on the output cell and choose "Disable Scrolling for outputs" to disengage the right hand slider

In [ ]:
#FFHQ single transformation (original images side by side with transformed images, sample size 10)  - evaluate blond hair
interactive_display_images("./ffhq/images",give_paths("./ffhq/reduced/images/single transformation/images"), 1, 200, "values_ffhq_ST1.csv")

In [ ]:
#CelebA multi transformation (original images side by side with transformed images, sample size 10) - evaluate rejuvenation
interactive_display_images("./celeba_Orig/images",give_paths("./celeba_Orig/reduced/images/multi transformation/images"), 0, 200,"values_CelebA_MT1.csv")

In [ ]:
#FFHQ multi transformation (original images side by side with transformed images, sample size 10) - evaluate rejuvenation
interactive_display_images("./ffhq/images",give_paths("./ffhq/reduced/images/multi transformation/images"), 1, 200, "values_ffhq_ST1.csv")

In [ ]:
# Load the 2 CSV files (scores by each collaborator) into DataFrames
df1 = pd.read_csv("values_CelebA_ST1.csv")
df2 = pd.read_csv("values_CelebA_ST2.csv")

# Assuming both DataFrames have a common column named 'id'
values _CelebA_ST = pd.concat([df1, df2], ignore_index=True)

# Load the 2 CSV files (scores by each collaborator) into DataFrames
df1 = pd.read_csv("values_ffhq_ST1.csv")
df2 = pd.read_csv("values_ffhq_ST2.csv")

# Assuming both DataFrames have a common column named 'id'
values _ffhq_ST = pd.concat([df1, df2], ignore_index=True)

# Load the 2 CSV files (scores by each collaborator) into DataFrames
df1 = pd.read_csv("values_CelebA_MT1.csv")
df2 = pd.read_csv("values_CelebA_MT2.csv")

# Assuming both DataFrames have a common column named 'id'
values _CelebA_MT = pd.concat([df1, df2], ignore_index=True)

# Load the 2 CSV files (scores by each collaborator) into DataFrames
df1 = pd.read_csv("values_ffhq_MT1.csv")
df2 = pd.read_csv("values_ffhq_MT2.csv")

# Assuming both DataFrames have a common column named 'id'
values _ffhq_MT = pd.concat([df1, df2], ignore_index=True)

Qualitative analysis - calculations

In [ ]:
print("CelebA single transformation to blond hair")
# Calculate the mean
mean_score = np.mean(values_CelebA_ST)

# Calculate the standard deviation
std_deviation = np.std(values_CelebA_ST)

print(f"Mean: {mean_score}")
print(f"Standard Deviation: {std_deviation}")

In [ ]:
print("FFHQ single transformation to blond hair")
# Calculate the mean
mean_score = np.mean(values_ffhq_ST)

# Calculate the standard deviation
std_deviation = np.std(values_ffhq_ST)

print(f"Mean: {mean_score}")
print(f"Standard Deviation: {std_deviation}")

In [ ]:
print("CelebA multi transformation to young")
# Calculate the mean
mean_score = np.mean(values_CelebA_MT)

# Calculate the standard deviation
std_deviation = np.std(values_CelebA_MT)

print(f"Mean: {mean_score}")
print(f"Standard Deviation: {std_deviation}")

In [ ]:
print("FFHQ multi transformation to blond hair")
# Calculate the mean
mean_score = np.mean(values_ffhq_MT)

# Calculate the standard deviation
std_deviation = np.std(values_ffhq_MT)

print(f"Mean: {mean_score}")
print(f"Standard Deviation: {std_deviation}")

Qualitative analysis - histograms

In [ ]:
import matplotlib.pyplot as plt

# Plot histograms
fig, axes = plt.subplots(2, 2, figsize=(12, 10), sharex=True, sharey=True)

# CelebA Single Transformation (ST)
axes[0, 0].hist(values_CelebA_ST, bins=10, range=(0, 10), color='blue', alpha=0.7, edgecolor='black')
axes[0, 0].set_title('CelebA - Blond Hair Transformation (ST)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(axis='y', linestyle='--', alpha=0.7)

# FFHQ Single Transformation (ST)
axes[0, 1].hist(values_ffhq_ST, bins=10, range=(0, 10), color='green', alpha=0.7, edgecolor='black')
axes[0, 1].set_title('FFHQ - Blond Hair Transformation (ST)')
axes[0, 1].grid(axis='y', linestyle='--', alpha=0.7)

# CelebA Multiple Transformation (MT)
axes[1, 0].hist(values_CelebA_MT, bins=10, range=(0, 10), color='purple', alpha=0.7, edgecolor='black')
axes[1, 0].set_title('CelebA - Blond Hair + Young Transformation (MT)')
axes[1, 0].set_xlabel('Scores')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(axis='y', linestyle='--', alpha=0.7)

# FFHQ Multiple Transformation (MT)
axes[1, 1].hist(values_ffhq_MT, bins=10, range=(0, 10), color='orange', alpha=0.7, edgecolor='black')
axes[1, 1].set_title('FFHQ - Blond Hair + Young Transformation (MT)')
axes[1, 1].set_xlabel('Scores')
axes[1, 1].grid(axis='y', linestyle='--', alpha=0.7)

# Adjust layout and display
plt.tight_layout()
plt.show()


The histograms will reveal:
Distribution shape (e.g., normal, skewed).
Variability in ratings (spread of scores).
Differences in scoring patterns between single and multiple transformations or between datasets.